In [ ]:
!pip install tensorflow

In [366]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

cleaning + preprocessing data 

In [367]:
stocks_df = pd.read_csv('stocks.csv')

/var/folders/v6/lhv05hmx5v94886x_jxd39s00000gn/T/ipykernel_20597/1353134829.py:1: DtypeWarning: Columns (9,49,57) have mixed types. Specify dtype option on import or set low_memory=False.
  stocks_df = pd.read_csv('stocks.csv')


In [369]:
missing_percentage = stocks_df.isnull().sum() * 100 / len(stocks_df)
columns_to_drop = missing_percentage[missing_percentage > 90].keys()
stocks_df = stocks_df.drop(columns=columns_to_drop, axis=1)
stocks_df = stocks_df.dropna()

In [371]:
def sequences(data, length=10): 
    X, y = [], []

    for i in range(len(data) - length): 
        X.append(data.iloc[i:(i + length)].values)
        y.append(data.iloc[i + length]['PRC'])
    
    return np.array(X), np.array(y)

In [373]:
def prepare_data(df, train_start, train_end, test_start, test_end): 
    print(df.columns)
    df['date'] = pd.to_datetime(df['date'])

    df = df.sort_values(by=['date'])
    print('sorted')
    

    train_data = df[
        (df['date'] >= train_start) & 
        (df['date'] <= train_end)
    ].copy()

    test_data = df[
        (df['date'] >= test_start) & 
        (df['date'] <= test_end)
    ].copy()

    print('train and test')

    X_train = 
    y_train =
    X_test = 
    y_test = 

    print('sequences')

    return X_train, y_train, X_test, y_test

In [374]:
train_start = '2019-01-02'
train_end = '2022-12-30'
test_start = '2023-01-03'
test_end = '2023-12-29'

In [375]:
X_train, y_train, X_test, y_test = prepare_data(
    stocks_df, 
    train_start,
    train_end,
    test_start,
    test_end
)

Index(['PERMNO', 'date', 'SHRCD', 'EXCHCD', 'SICCD', 'NCUSIP', 'TICKER',
       'SHRCLS', 'TSYMBOL', 'NAICS', 'PRIMEXCH', 'HSICCD', 'CUSIP', 'HSICMG',
       'HSICIG', 'BIDLO', 'ASKHI', 'PRC', 'VOL', 'RET', 'BID', 'ASK', 'SHROUT',
       'CFACPR', 'CFACSHR', 'OPENPRC', 'NUMTRD', 'RETX', 'vwretd', 'vwretx',
       'ewretd', 'ewretx', 'sprtrn'],
      dtype='object')
sorted
train and test
sequences


CNN-LSTM hybrid model for stock price prediction

In [359]:
scaler = StandardScaler()

X_train_scaled = X_train.copy(deep = True)
X_test_scaled = X_test.copy(deep = True)

scaler.fit(X_train_scaled)

X_train_scaled = scaler.transform(X_train_scaled)
X_test_scaled = scaler.transform(X_test_scaled)

KeyError: "['date'] not in index"

input layer 
one-dimensional convolution layer 
pooling layer 
LSTM hidden layer 
full connection layer 

In [360]:
X_train_reshaped = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1])

In [361]:
model = models.Sequential()
model.add(layers.Conv1D(filters=64, kernel_size=3, padding='same', activation='tanh', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(layers.MaxPooling1D(pool_size=2, padding='same'))
model.add(LSTM(units=100, activation='tanh'))
model.add(Dense(units=1))

/Users/monishakrothapalli/Documents/GitHub/playlistGen/.conda/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [362]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer, loss='mean_absolute_error', metrics=['mean_absolute_error'])

In [363]:
history = model.fit(X_train_reshaped, y_train, epochs=100, batch_size=64, validation_split=0.1)

train_accuracy = history.history['mean_absolute_error']

for epoch, accuracy in enumerate(train_accuracy):
    print(f"Epoch {epoch + 1}: mean_absolute_error = {accuracy:.4f}")

Epoch 1/100
 1/12 ━━━━━━━━━━━━━━━━━━━━ 7s 669ms/step - loss: 258.5034 - mean_absolute_error: 258.5034

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 83
'y' sizes: 73


In [315]:
X_test_scaled_array = X_test_scaled.to_numpy()
X_test_scaled_reshaped = X_test_scaled_array.reshape(X_test_scaled_array.shape[0], 1, X_test_scaled_array.shape[1])

predictions = model.predict(X_test_scaled_reshaped)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [316]:
mae = mean_absolute_error(y_test['PRC'], predictions)
rmse = np.sqrt(mean_squared_error(y_test['PRC'], predictions))
r2 = r2_score(y_test['PRC'], predictions)

print(f'MAE: {mae}, RMSE: {rmse}, R2: {r2}')

MAE: 199.0390944218925, RMSE: 225.26410767150287, R2: -7.231628078127789
